In [1]:
import mysql.connector
import pandas as pd

mydb = mysql.connector.connect(
    host= "localhost",
    user= "root",
    password= "root",
    database= "st"
)
sql_query = "SELECT * FROM coord"

# Execute the SQL query
cursor = mydb.cursor()
cursor.execute(sql_query)

rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]

df = pd.DataFrame(rows, columns=column_names)
cursor.close()
mydb.close()


Skipped 200 to 204 Due to some error in 203 please check even 249 since its in another state. Skippped 1002

Haven't done from tid = df['trip_id'].unique()[1500:1600] 

In [12]:
import requests
import json
import mysql.connector
import numpy as np

def create_pairs(data):
    pairs = []
    first_lat_lng = f"{data.iloc[0]['longitude']},{data.iloc[0]['latitude']}"
    pairs.append(first_lat_lng)
    for i in range(4, len(data), 5):  # Starting from index 9, select every 10th latitude and longitude
        lat_lng = f"{data.iloc[i]['longitude']},{data.iloc[i]['latitude']}"
        pairs.append(lat_lng)
    last_lat_lng = f"{data.iloc[-1]['longitude']},{data.iloc[-1]['latitude']}"
    pairs.append(last_lat_lng)
    return pairs

def cpmini(data):
    pairs = []
    first_lat_lng = f"{data.iloc[0]['longitude']},{data.iloc[0]['latitude']}"
    
    pairs.append(first_lat_lng)
    for i in range(0, len(data)):  # Starting from index 0
        lat_lng = f"{data.iloc[i]['longitude']},{data.iloc[i]['latitude']}"
        pairs.append(lat_lng)
    last_lat_lng = f"{data.iloc[-1]['longitude']},{data.iloc[-1]['latitude']}"
    # if last_lat_lng not in pairs:
    pairs.append(last_lat_lng)
    return pairs

def send_osrm_request(pairs, trip_id, speed, recorded):
    radiuses = ['49'] * len(pairs)
    url = f"http://localhost:5000/match/v1/bicycle/{';'.join(pairs)}?overview=full&radiuses={';'.join(radiuses)}&geometries=geojson&annotations=true"
    response = requests.get(url)
    if response.status_code == 200:
        result = response.json() 
    
        node_id = ""
    
        for match in result['matchings']:
            for leg in match['legs']:
                for n_id in leg['annotation']['nodes']:
                    if node_id:
                        node_id += ","
                    node_id += str(n_id)
            # for point in match['geometry']['coordinates']:
        matched_coordinates.append({'trip_id': trip_id, 'response': result, 'speed': speed, 'recorded': recorded, 'node_id':node_id})
    else:
        print("Error:", response.status_code)
    return matched_coordinates

matched_coordinates = []
# batch_data = df[df['trip_id'] == 2]

# pairs = create_pairs(batch_data)
# matched_coordinates_response = send_osrm_request(pairs)

tid = df['trip_id'].unique()[2400:2700] 
for i in tid:
    # batch_data = df[df['trip_id'] == i]
    batch_data = df[df['trip_id'] == i]

    if len(batch_data) == 1:

        if len(batch_data) == 1:
            rec = str(batch_data.iloc[0]['recorded'])  # Get the recorded time
            spd = str(batch_data.iloc[0]['speed'])
            node_id = "NA"
            resp = f"{batch_data.iloc[0]['latitude']},{batch_data.iloc[0]['longitude']}"
            matched_coordinates.append({'trip_id': trip_id, 'response': resp, 'speed': spd, 'recorded': rec, 'node_id':node_id})
            print("Response from OSRM API:", matched_coordinates)

    else:

        if len(batch_data) <= 9:
                splitted_dfs = batch_data
                print(f"DataFrames with less than 9 value", splitted_dfs)
                tstmp = ""
                sr = ""
                for k in range(0, len(splitted_dfs)):

                        recorded_time = str(splitted_dfs.iloc[k]['recorded'])  # Get the recorded time
                        if tstmp:  # Check if the string is not empty
                            tstmp += ","  # Add comma separator if needed
                        tstmp += recorded_time

                        speedrng = str(splitted_dfs.iloc[k]['speed'])  # Get the recorded time
                        if sr:  # Check if the string is not empty
                            sr += ","  # Add comma separator if needed
                        sr += speedrng
                
                trip_id = f"{splitted_dfs.iloc[0]['trip_id']}"
                pairs = cpmini(splitted_dfs)
                matched_coordinates = send_osrm_request(pairs, trip_id=trip_id, speed=sr, recorded=tstmp)
        else:

            if 100 >= len(batch_data) > 9:
                sdfs = np.array_split(batch_data, 1)

            elif 500 >= len(batch_data) > 100:
                sdfs = np.array_split(batch_data, 2)

            elif 1000 >= len(batch_data) > 500:
                sdfs = np.array_split(batch_data, 3)

            elif 5000 >= len(batch_data) > 1000:
                sdfs = np.array_split(batch_data, 15)

            elif 10000 >= len(batch_data) > 5000:
                sdfs = np.array_split(batch_data, 30)

            elif 15000 >= len(batch_data) > 10000:
                sdfs = np.array_split(batch_data, 40)

            else:
                sdfs = np.array_split(batch_data, 70)

            for j, df_part in enumerate(sdfs):
                
                print(f"DataFrame {j+1}:\n{df_part}\n")
                timestmp = ""
                speedrange = ""
                for k in range(0, len(df_part), 5):

                        recorded_time = str(df_part.iloc[k]['recorded'])  # Get the recorded time
                        if timestmp:  # Check if the string is not empty
                            timestmp += ","  # Add comma separator if needed
                        timestmp += recorded_time

                        speedrng = str(df_part.iloc[k]['speed'])  # Get the recorded time
                        if speedrange:  # Check if the string is not empty
                            speedrange += ","  # Add comma separator if needed
                        speedrange += speedrng
                
                trip_id = f"{df_part.iloc[0]['trip_id']}"
                # pairs = cp(df_part)
                # matched_coordinates = sor(pairs, trip_id=trip_id, speed=speedrange, recorded=timestmp)
                pairs = create_pairs(df_part)
                matched_coordinates = send_osrm_request(pairs, trip_id=trip_id, speed=speedrange, recorded=timestmp)
                # print("Response from OSRM API:", matched_coordinates_response)
                # print("Response from OSRM API:", matched_coordinates)

print("Response from OSRM API:", matched_coordinates)


DataFrame 1:
         trip_id            recorded   latitude  longitude    altitude  speed  \
2082813     2410 2019-05-23 13:29:26  33.770039 -84.306710  312.195278   0.76   
2082814     2410 2019-05-23 13:29:27  33.770025 -84.306754  312.558681   0.76   
2082815     2410 2019-05-23 13:29:28  33.769983 -84.306807  310.482693   1.12   
2082816     2410 2019-05-23 13:29:29  33.770118 -84.306758  313.890224   0.00   
2082817     2410 2019-05-23 13:29:30  33.770166 -84.306650  314.771450   0.00   
...          ...                 ...        ...        ...         ...    ...   
2083122     2410 2019-05-23 13:38:43  33.758823 -84.318813  299.497524   6.22   
2083123     2410 2019-05-23 13:38:44  33.758758 -84.318824  299.242886   6.98   
2083124     2410 2019-05-23 13:38:45  33.758681 -84.318809  298.497097   7.77   
2083125     2410 2019-05-23 13:38:46  33.758613 -84.318800  297.708828   7.81   
2083126     2410 2019-05-23 13:38:47  33.758542 -84.318780  297.381435   8.02   

         hAccu

In [ ]:
import mysql.connector
import pandas as pd

mydb = mysql.connector.connect(
    host= "localhost",
    user= "root",
    password= "root",
    database= "st"
)

sql_query = "SELECT * FROM osr WHERE trip_id BETWEEN 1299 AND 1302"

# Execute the SQL query
cursor = mydb.cursor()
cursor.execute(sql_query)

# Fetch the rows and column names
rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]

# Create a DataFrame from the fetched rows and column names
df = pd.DataFrame(rows, columns=column_names)

# Print the DataFrame
print(df)
cursor.close()
mydb.close()

In [ ]:
batch_data = df[df['trip_id'] == 2006]
print(batch_data)

unique_count = df['trip_id'].nunique()
print("Number of unique values:", unique_count)
# if len(batch_data) == 1:
#     rec = str(batch_data.iloc[0]['recorded'])  # Get the recorded time
#     spd = str(batch_data.iloc[0]['speed'])
#     node_id = "NA"
#     resp = f"{batch_data.iloc[0]['latitude']},{batch_data.iloc[0]['longitude']}"
#     # matched_coordinates.append({'trip_id': trip_id, 'response': resp, 'speed': spd, 'recorded': rec, 'node_id':node_id})
#     print("Response from OSRM API:", resp,spd,rec, node_id)

In [ ]:
batch_data = df[df['trip_id'] == 203]

print(batch_data)

In [ ]:
for record in matched_coordinates:
    trip_id = record['trip_id']
    response = json.dumps(record['response'])  # Convert response to JSON string
    speed = record['speed']
    recorded = record['recorded']
    node_id = record['node_id']
    print(trip_id, response, speed, recorded, node_id)

In [5]:

import mysql.connector

mydb = mysql.connector.connect(
    host= "localhost",
    user= "root",
    password= "root",
    database= "st"
)

create_table_query = """
CREATE TABLE IF NOT EXISTS osr (
    trip_id INT,
    response LONGTEXT,
    speed TEXT,
    recorded TEXT,
    node_id TEXT
)
"""

# Execute the table creation query
cursor = mydb.cursor()
cursor.execute(create_table_query)

insert_query = "INSERT INTO osr (trip_id, response, speed, recorded, node_id) VALUES (%s, %s, %s, %s, %s)"

# Iterate over matched_coordinates and insert each record into the table
for record in matched_coordinates:
    response = json.dumps(record['response'])  # Convert response to JSON string
    cursor.execute(insert_query, (record['trip_id'], response, record['speed'], record['recorded'], record['node_id']))

# Commit changes and close the cursor and MySQL connection
mydb.commit()
cursor.close()
mydb.close()


In [11]:
# -----------------------------updating table--------------------------------


import mysql.connector

mydb = mysql.connector.connect(
    host= "localhost",
    user= "root",
    password= "root",
    database= "st"
)

# Execute the table creation query
cursor = mydb.cursor()


insert_query = "INSERT INTO osr (trip_id, response, speed, recorded, node_id) VALUES (%s, %s, %s, %s, %s)"

# Iterate over matched_coordinates and insert each record into the table
for record in matched_coordinates:
    response = json.dumps(record['response'])  # Convert response to JSON string
    cursor.execute(insert_query, (record['trip_id'], response, record['speed'], record['recorded'], record['node_id']))

# Commit changes and close the cursor and MySQL connection
mydb.commit()
cursor.close()
mydb.close()


In [ ]:
# Update Table

import mysql.connector

mydb = mysql.connector.connect(
    host= "localhost",
    user= "root",
    password= "root",
    database= "st"
)

insert_query = """
INSERT INTO osrmresp (response) 
VALUES (%s) 
ON DUPLICATE KEY UPDATE response = VALUES(response)
"""

# Execute the insertion query
cursor = mydb.cursor()
cursor.execute(insert_query, (json_response,))

mydb.commit()
cursor.close()
mydb.close()

In [6]:
import mysql.connector

mydb = mysql.connector.connect(
    host= "localhost",
    user= "root",
    password= "root",
    database= "st"
)

with mydb.cursor() as cursor:
    cursor.execute("DROP TABLE IF EXISTS osr")

# Commit changes and close the MySQL connection
mydb.commit()
mydb.close()

In [19]:
print("Response from OSRM API:", json_response)

Response from OSRM API: {"code": "Ok", "matchings": [{"confidence": 0.906154, "geometry": {"coordinates": [[-84.364281, 33.77768], [-84.364234, 33.777681], [-84.364234, 33.777609], [-84.364234, 33.777681], [-84.364281, 33.77768], [-84.364281, 33.77768], [-84.364282, 33.777393], [-84.364283, 33.777279], [-84.364283, 33.777279], [-84.364284, 33.777171], [-84.364284, 33.77701], [-84.364285, 33.776874], [-84.364286, 33.776746], [-84.364308, 33.776745], [-84.364464, 33.776744], [-84.364568, 33.776743], [-84.364568, 33.776743], [-84.364574, 33.776689], [-84.364742, 33.776687], [-84.365276, 33.776677], [-84.365289, 33.776677], [-84.365347, 33.776676], [-84.36515, 33.77587], [-84.364954, 33.775072], [-84.364863, 33.7747], [-84.364783, 33.774248], [-84.364696, 33.773752], [-84.364634, 33.773507], [-84.36462, 33.773451], [-84.364561, 33.773218], [-84.364412, 33.772632], [-84.364382, 33.772521], [-84.364281, 33.772143], [-84.364277, 33.772126], [-84.364229, 33.771991], [-84.364113, 33.77173], [-8